In [7]:
import requests
import pandas as pd
import lxml.html as lh

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

##### Hacemos la petición a la web a scrapear, la parseamos y seleccionamos todos los tag 'tr', ya que son el elemento en común que tienen todos los datos que necesitamos:

In [13]:
url = 'https://www.numbeo.com/cost-of-living/prices_by_country.jsp?displayCurrency=EUR&itemId=101&itemId=100&itemId=228&itemId=224&itemId=60&itemId=66&itemId=64&itemId=62&itemId=110&itemId=118&itemId=121&itemId=14&itemId=19&itemId=17&itemId=15&itemId=11&itemId=16&itemId=113&itemId=9&itemId=12&itemId=8&itemId=119&itemId=111&itemId=112&itemId=115&itemId=116&itemId=13&itemId=27&itemId=26&itemId=29&itemId=28&itemId=114&itemId=6&itemId=4&itemId=5&itemId=3&itemId=2&itemId=1&itemId=7&itemId=105&itemId=106&itemId=44&itemId=40&itemId=42&itemId=24&itemId=20&itemId=18&itemId=109&itemId=108&itemId=107&itemId=206&itemId=25&itemId=32&itemId=30&itemId=33'

page = requests.get(url)

doc = lh.fromstring(page.content)

elements = doc.xpath('//tr')[2:]

##### Hacemos un pequeño check para comprobar que las filas seleccionadas tienen la misma longitud:

In [14]:
check = [len(elem) for elem in elements[:20]]

print(check)

[57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57]


##### Scrapeamos primero la cabecera y dejamos hueco para introducir posteriormente los datos de cada columna:

In [15]:
def getHeader(elem):
    col = elem.text_content()
    return ((col,[]))

columns = [getHeader(elem) for elem in elements[0]]

##### Scrapeamos el resto de los datos de la tabla y los añadimos al hueco que dejamos anteriormente junto a las columnas:

In [16]:
for i in range(1,len(elements)):
    element = elements[i]
    if len(element) != check[1]:    #check if same size
        break
    ind = 0                         #index of our column
    for k in element.iterchildren():
        row = k.text_content()
        if ind > 1:
            try:
                row = float(row)
            except:
                pass
        columns[ind][1].append(row)
        ind +=1

##### Hacemos otro check para comprobar si todas las columnas tienen la misma cantidad de datos:

In [17]:
check2 = [len(col) for (head,col) in columns]

print(check2)

[101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101]


##### Creamos el DataFrame:

In [18]:
elements_dict = {title:column for (title,column) in columns}

df = pd.DataFrame(elements_dict)

df.head()

,Rank,Country,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Coke/Pepsi (0.33 liter bottle),Water (0.33 liter bottle),"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Local Cheese (1kg),Water (1.5 liter bottle),Bottle of Wine (Mid-Range),Domestic Beer (0.5 liter bottle),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),"Chicken Breasts (Boneless, Skinless), (1kg)",Monthly Pass (Regular Price),Gasoline (1 liter),Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car),Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,"Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment",1 min. of Prepaid Mobile Tariff Local (No Discounts or Plans),"Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)","Fitness Club, Monthly Fee for 1 Adult",Tennis Court Rent (1 Hour on Weekend),"Cinema, International Release, 1 Seat",1 Pair of Jeans (Levis 501 Or Similar),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",Taxi Start (Normal Tariff),Taxi 1km (Normal Tariff),Taxi 1hour Waiting (Normal Tariff),Apples (1kg),Oranges (1kg),Potato (1kg),Lettuce (1 head),Cappuccino (regular),"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat),Toyota Corolla 1.6l 97kW Comfort (Or Equivalent New Car),"Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child","International Primary School, Yearly for 1 Child"
0,,Kazakhstan,4.75,23.75,4.28,1.19,1.16,0.45,0.31,0.70,0.24,0.83,5.41,0.44,4.75,0.60,0.95,0.21,3.07,16.63,0.39,15439.64,266.08,171.28,538.43,331.95,49.38,0.03,10.40,37.80,9.61,3.56,47.56,33.38,72.12,80.11,959.03,655.17,376.99,15.51,1.19,0.24,2.85,1.12,1.39,0.38,0.89,1.66,0.76,1.16,1.21,0.31,5.04,17986.69,142.99,6676.44
1,,Paraguay,3.45,20.72,3.52,0.83,1.10,0.72,0.42,0.70,0.52,1.47,4.48,0.55,5.53,0.73,1.38,0.48,2.43,21.02,0.83,11050.72,220.26,153.67,508.64,394.55,44.63,0.09,25.79,22.27,9.90,4.83,38.79,37.30,66.84,49.84,1134.65,526.24,316.20,13.94,0.69,1.66,6.91,1.55,0.82,0.85,0.35,1.44,0.62,1.34,0.62,0.88,4.89,19304.23,122.99,3072.71
2,,Portugal,7.50,30.00,6.00,1.80,1.86,1.26,0.92,0.63,1.09,1.69,6.82,0.55,4.00,0.94,5.00,1.52,5.13,35.00,1.54,25000.00,619.65,457.21,1112.40,768.25,96.61,0.17,31.86,32.75,11.04,7.00,80.06,29.17,70.90,79.53,2638.49,1565.04,830.15,2.26,3.45,0.47,14.80,1.53,1.15,0.87,0.89,1.32,0.88,1.39,1.10,0.95,9.39,24609.58,347.42,6959.47
3,,Syria,3.61,14.00,3.61,1.35,1.42,0.49,0.26,0.56,0.27,1.13,3.07,0.38,2.94,0.98,2.34,0.09,3.19,9.68,0.49,18027.28,176.54,82.25,350.18,196.74,17.15,0.02,30.94,13.07,4.73,4.38,14.08,22.22,65.82,36.54,1394.12,655.91,83.89,8.99,0.45,0.53,1.80,0.52,0.34,0.32,0.24,0.93,0.84,0.39,1.26,0.30,8.39,22578.32,52.33,843.80
4,,Greece,10.00,31.00,6.00,4.00,1.79,1.62,0.50,1.18,0.88,2.91,7.80,0.74,6.25,1.28,4.50,1.40,6.53,30.00,1.60,18000.00,323.04,270.85,544.25,465.08,145.67,0.37,30.81,32.34,15.32,8.00,70.93,29.51,73.05,88.41,1602.57,1410.51,708.74,4.66,3.50,0.80,11.81,1.39,1.02,0.83,0.71,3.01,1.55,1.32,1.49,0.82,8.86,18613.72,321.70,6615.96


##### Exportamos fichero a formato .csv para posteriormente tratarlo:

In [70]:
df.to_csv('INPUT/cost_of_life_by_country.csv')